# Run Functions to Add Information to Projects

In [1]:
# ! pip install nltk

In [1]:

import pandas as pd
from siuba import *

from calitp_data_analysis.sql import to_snakecase

import _data_utils
import _script_utils

In [2]:
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', None)

## Read in Data and function development

In [3]:
GCS_FILE_PATH  = 'gs://calitp-analytics-data/data-analyses/dla/dla-iija'

In [4]:
### replace the file path with the new data file
### Latest is from 6/29/23
proj = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/FMIS_Projects_Universe_IIJA_Reporting_062923_ToDLA.xlsx",))

### last data export 3/9/2023
# proj = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/IIJA_FMIS_AllProject_20230309_ToDLA.xlsx"))
                       


In [5]:

# df = utils.read_data_all()
proj = _data_utils.add_new_codes(proj)
    
## function that adds known agency name to df 
df = _script_utils.identify_agency(proj, 'summary_recipient_defined_text_field_1_value')

In [6]:
df.sample(3)

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,project_status_description,project_description,improvement_type,improvement_type_description,total_cost_amount,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name
267,2022-05-02,Y001,National Highway Performance Program (NHPP),P395348,0900000030S,IN INYO COUNTY NEAR OLANCHA AND CARTAGO FROM 1.4 MILES SOUTH OF LOS ANGELES AQUEDUCT BRIDGE TO 0.2 MILE SOUTH OF ASH CREEK BRIDGE. CONSTRUCT 4 LANE E,27,Cong Dist 8,Active,ON STATE ROUTE: 395. IN INYO COUNTY NEAR OLANCHA AND CARTAGO FROM 1.4 MILES SOUTH OF LOS ANGELES AQUEDUCT BRIDGE TO 0.2 MILE SOUTH OF ASH CREEK BRIDGE. CONSTRUCT 4 LANE EXPRESSWAY.,8,Bridge New Construction,3645800.00,3227500.00,S NON-MPO,NaN,California,9.0,Inyo County,NaN,NaN
468,2022-09-12,Y240,Surface Transportation Block Grant,5027023,0422000486L,HEALDSBURG AVENUE BETWEEN POWELL AVENUE AND PASSALAQUA ROAD COMPLETE STREET IMPROVEMENTS,97,Cong Dist 2,Active,HEALDSBURG AVENUE BETWEEN POWELL AVENUE AND PASSALAQUA ROAD COMPLETE STREET IMPROVEMENTS,15,Preliminary Engineering,690791.00,600000.00,L5920MTC,5920.0,Sonoma County,4.0,Sonoma County,Metropolitan Transportation Commission,Metropolitan Transportation Commission
553,2022-06-21,Y001,National Highway Performance Program (NHPP),2801153,0416000024S,"SANTA CLARA COUNTY IN AND NEAR SAN JOSE FROM S280-N101/101 SEPARATION TO ROUTE 17/280 SEPERATION MINOR CONCRETE, MGS RAILING, HMA AND HMA DIKE.",85,Cong Dist 19,Active,"ON STATE ROUTE: 280. SANTA CLARA COUNTY IN AND NEAR SAN JOSE FROM S280-N101/101 SEPARATION TO ROUTE 17/280 SEPERATION MINOR CONCRETE, MGS RAILING, HMA AND HMA DIKE.",17,Construction Engineering,368336.77,337246.65,S MTC,NaN,California,4.0,Santa Clara County,NaN,NaN


## Test & Export

In [17]:
### run the data through the rest of the script
### return a dataset that is aggregated at the project and program code

agg = _script_utils.get_clean_data(df, full_or_agg = 'agg')

/home/jovyan/data-analyses/dla/iija/_script_utils.py:465: FutureWarning: The default value of regex will change from True to False in a future version.
  df['implementing_agency_locode'] = df['implementing_agency_locode'].str.replace('.0', '')


In [18]:
agg.sample(5)

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,program_code,program_code_description,recipient_project_number,improvement_type,improvement_type_description,old_project_title_desc,obligations_amount,congressional_district,district,county_code,county_name,county_name_abbrev,implementing_agency_locode,rtpa_name,mpo_name,new_project_title,new_description_col
477,2022-09-15,5951152,Santa Barbara County,L5951SBCAG,Y233,Surface Transportation Block Grant,0513000106L,15|17|47,Preliminary Engineering|Construction Engineering|Bridge Preventive Maintenance,ALAMO PINTADO BRIDGE #51C0081 ON ALAMO PINTADO RD @ ALAMO PINTADO CRK LOS OLIVOS REPAIR & CONSTRUCTION OF SCOUR COUNTERMEASURE (TC),140166,|24|,|05|,83,Santa Barbara County,|SB|,5951,Santa Barbara County Association of Governments,Santa Barbara County Association Of Governments,Construct Erosion Countermeasures in Santa Barbara County,"Construct Erosion Countermeasures in Santa Barbara County, part of the program(s) Surface Transportation Block Grant. (Federal Project ID: 5951152)."
702,2023-01-20,5390006,Portola Valley,L5390MTC,Y230,Surface Transportation Block Grant,0423000011L,5,4R - Maintenance Resurfacing,"WESTRIDGE DRIVE FROM APPROXIMATELY ALPINE ROAD TO CERVANTES ROAD PAVEMENT PRESERVATION INCLUDING PAVEMENT GRINDING, BASE REPAIR, SLURRY SEAL, THERMOP",201000,|18|,|04|,81,San Mateo County,|SM|,53,Metropolitan Transportation Commission,Metropolitan Transportation Commission,Maintenance Resurfacing in Portola Valley,"Maintenance Resurfacing in Portola Valley, part of the program(s) Surface Transportation Block Grant. (Federal Project ID: 5390006)."
997,2023-05-02,5361023,Novato,L5361MTC,Y001,National Highway Performance Program (NHPP),0400002094L,13|17,Bridge Rehabilitation - Added Capacity|Construction Engineering,GRANT AVENUE OVER NOVATO CREEK. BR.# 27C0021 BRIDGE REHABILITATION AND SCOUR MITIGATION,3219836,|02|,|04|,41,Marin County,|MRN|,5361,Metropolitan Transportation Commission,Metropolitan Transportation Commission,Erosion Countermeasures in Novato,"Erosion Countermeasures in Novato, part of the program(s) National Highway Performance Program (NHPP). (Federal Project ID: 5361023)."
36,2022-02-09,32L0517,Contra Costa County,L5928MTC,ER01,Emergency Relieve Funding,0418000419L,1|17,New Construction Roadway|Construction Engineering,"205 MILES NORTHWEST OF CANYON RD AT THE FIRST HAIRPIN TURN IN UNINCORPORATED ORINDA. REPLACE THE FIRST 50"" OF DAMAGED 72"" CORRUGATED METAL PIPE CULVE",374216,|05|,|04|,13,Contra Costa County,|CC|,5928,Metropolitan Transportation Commission,Metropolitan Transportation Commission,New Construction Roadway in Contra Costa County,"New Construction Roadway in Contra Costa County, part of the program(s) Emergency Relieve Funding. (Federal Project ID: 32L0517)."
394,2022-09-01,000C529,California,S SCAG,YS30,Highway Safety Improvement Program (HSIP),0812000120S,17|21,Construction Engineering|Safety,IN THE CITY OF BEAUMONT ON ROUTE 10 FROM 6TH STREET ON-RAMP TO I-10 WESTBOUND AND ON ROUTE 210 IN SAN BERNARDINO FROM 0.1 MI TO 0.2 MI E/O DEL ROSA A,1060200,|36|31|,|08|,65|71,Riverside County|San Bernardino County,|RIV|SBD|,nan,None,None,Ramp in Riverside County & San Bernardino County,"Ramp in Riverside County & San Bernardino County, part of the program(s) Highway Safety Improvement Program (HSIP). (Federal Project ID: 000C529)."


In [19]:
## when grouping by funding program (pne project can have multiple rows), len is 1612
len(agg)

1465

In [23]:
df.project_number.nunique()

1465

In [21]:
agg>>count(_.project_number)>>filter(_.n>1)>>arrange(-_.n)

,project_number,n


In [22]:
agg>>filter(_.project_number=='5004049')

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,program_code,program_code_description,recipient_project_number,improvement_type,improvement_type_description,old_project_title_desc,obligations_amount,congressional_district,district,county_code,county_name,county_name_abbrev,implementing_agency_locode,rtpa_name,mpo_name,new_project_title,new_description_col
1372,2023-06-26,5004049,San Diego,L5004SANDAG,Y001|Y110|Y908|Y909,National Highway Performance Program (NHPP)|Bridge Formula Program|Bridge Replacement and Rehabilitation Program,11955780L,10|17,Bridge Replacement - Added Capacity|Construction Engineering,"WEST MISSION BAY DRIVE OVER THE SAN DIEGO RIVER BRIDGE REPLACEMENT, BR. NO. 57C-0023",69715548,|52|,|11|,73,San Diego County,|SD|,04,San Diego Association of Governments,San Diego Association Of Governments,Replace Bridge in San Diego,"Replace Bridge in San Diego, part of the program(s) National Highway Performance Program (NHPP), and the Bridge Formula Program, and the Bridge Replacement and Rehabilitation Program. (Federal Project ID: 5004049)."


In [24]:
### pretty print the column names

agg = _script_utils.title_column_names(agg)

In [25]:
#### export new file to the GCS bucket
# agg.to_csv(f"{GCS_FILE_PATH}/FMIS_Projects_Universe_IIJA_Reporting_091423_rerun.csv")

### old export
# agg.to_csv(f"{GCS_FILE_PATH}/FMIS_Projects_Universe_IIJA_Reporting_3-9-23_agg.csv")


In [13]:
# ###test full. PASS title, no concat and 1241 rows
# full = _script_utils.get_clean_data(df, full_or_agg = 'full')